In [1]:
from unidecode import unidecode
import matplotlib.pyplot as plt
from zipfile import ZipFile
import mplfinance as mpf
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import json

## API CONFIG

In [3]:
API_KEY = '01NKJDZHS13H8U13'

base_url = 'https://www.alphavantage.co/query'
param_function = '?function=TIME_SERIES_DAILY'
param_symbol   = '&symbol=PETR4.SAO'
param_output   = '&outputsize=full'
param_api_key  = f'&apikey={API_KEY}'

final_url = base_url + param_function + param_symbol + param_output + param_api_key
print(final_url)

 r = requests.get(final_url)

last_update = json.loads(r.content)['Meta Data']['3. Last Refreshed']
time_series = json.loads(r.content)['Time Series (Daily)']

df = pd.DataFrame.from_dict(time_series).T.reset_index()
df.columns = ['day','open','high','low','close','volume']

df['day'] = pd.to_datetime(df['day'])
df = df.set_index('day')

df['open'] = df['open'].astype(float)
df['high'] = df['high'].astype(float)
df['low'] = df['low'].astype(float)
df['close'] = df['close'].astype(float)
df['volume'] = df['volume'].astype(float)

df = df[df.index >= '2019-01-01']

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=PETR4.SAO&outputsize=full&apikey=01NKJDZHS13H8U13


## GET SETORIAL DATA

In [2]:
df_setorial = pd.read_csv('../data/b3_setorial.csv', sep=';', encoding='latin')

# for index in range(len(df_setorial['segmento'].unique())):
#     print(f"'{df_setorial['segmento'].unique()[index]}' :\
#  '{unidecode(df_setorial['segmento'].unique()[index]).lower().replace(' ','_').replace(',','')}',")


# SETOR
setor_map = {
    'Petróleo, Gás e Biocombustíveis' : 'petroleo_gas_e_biocombustiveis',
    'Materiais Básicos' : 'materiais_basicos',
    'Bens Industriais' : 'bens_industriais',
    'Consumo não Cíclico' : 'consumo_nao_ciclico',
    'Consumo Cíclico' : 'consumo_ciclico',
    'Saúde' : 'saude',
    'Tecnologia da Informação' : 'tecnologia_da_informacao',
    'Comunicações' : 'comunicacoes',
    'Utilidade Pública' : 'utilidade_publica',
    'Financeiro' : 'financeiro',
    'Outros' : 'outros'    
}
df_setorial['setor'] = df_setorial['setor'].map(setor_map)
setor_dummies = pd.get_dummies(df_setorial.setor, prefix='setor', prefix_sep='.').drop(columns='setor.outros')
df_setorial = pd.concat([df_setorial, setor_dummies], axis=1).drop(columns='setor')

# SUBSETOR
subseto_map = {
    'Petróleo, Gás e Biocombustíveis' : 'petroleo_gas_e_biocombustiveis',
    'Mineração' : 'mineracao',
    'Siderurgia e Metalurgia' : 'siderurgia_e_metalurgia',
    'Químicos' : 'quimicos',
    'Madeira e Papel' : 'madeira_e_papel',
    'Embalagens' : 'embalagens',
    'Materiais Diversos' : 'materiais_diversos',
    'Construção e Engenharia' : 'construcao_e_engenharia',
    'Material de Transporte' : 'material_de_transporte',
    'Máquinas e Equipamentos' : 'maquinas_e_equipamentos',
    'Transporte' : 'transporte',
    'Serviços Diversos' : 'servicos_diversos',
    'Comércio' : 'comercio',
    'Agropecuária' : 'agropecuaria',
    'Alimentos Processados' : 'alimentos_processados',
    'Bebidas' : 'bebidas',
    'Produtos de Uso Pessoal e de Limpeza' : 'produtos_de_uso_pessoal_e_de_limpeza',
    'Comércio e Distribuição' : 'comercio_e_distribuicao',
    'Construção Civil' : 'construcao_civil',
    'Tecidos, Vestuário e Calçados' : 'tecidos_vestuario_e_calcados',
    'Utilidades Domésticas' : 'utilidades_domesticas',
    'Automóveis e Motocicletas' : 'automoveis_e_motocicletas',
    'Hoteis e Restaurantes' : 'hoteis_e_restaurantes',
    'Viagens e Lazer' : 'viagens_e_lazer',
    'Diversos' : 'diversos',
    'Medicamentos e Outros Produtos' : 'medicamentos_e_outros_produtos',
    'Serviços Médico - Hospitalares, Análises e Diagnósticos' : 'servicos_medico_-_hospitalares_analises_e_diagnosticos',
    'Equipamentos' : 'equipamentos',
    'Computadores e Equipamentos' : 'computadores_e_equipamentos',
    'Programas e Serviços' : 'programas_e_servicos',
    'Telecomunicações' : 'telecomunicacoes',
    'Mídia' : 'midia',
    'Energia Elétrica' : 'energia_eletrica',
    'Água e Saneamento' : 'agua_e_saneamento',
    'Gás' : 'gas',
    'Intermediários Financeiros' : 'intermediarios_financeiros',
    'Securitizadoras de Recebíveis' : 'securitizadoras_de_recebiveis',
    'Serviços Financeiros Diversos' : 'servicos_financeiros_diversos',
    'Previdência e Seguros' : 'previdencia_e_seguros',
    'Exploração de Imóveis' : 'exploracao_de_imoveis',
    'Holdings Diversificadas' : 'holdings_diversificadas',
    'Outros Títulos' : 'outros_titulos',
    'Outros' : 'outros'
}

df_setorial['subsetor'] = df_setorial['subsetor'].map(setor_map)
setor_dummies = pd.get_dummies(df_setorial.subsetor, prefix='subsetor', prefix_sep='.').drop(columns='subsetor.outros')
df_setorial = pd.concat([df_setorial, setor_dummies], axis=1).drop(columns='subsetor')

# SEGMENTO
segmento_map = {
    'Exploração, Refino e Distribuição' : 'exploracao_refino_e_distribuicao',
    'Equipamentos e Serviços' : 'equipamentos_e_servicos',
    'Minerais Metálicos' : 'minerais_metalicos',
    'Siderurgia' : 'siderurgia',
    'Artefatos de Ferro e Aço' : 'artefatos_de_ferro_e_aco',
    'Artefatos de Cobre' : 'artefatos_de_cobre',
    'Petroquímicos' : 'petroquimicos',
    'Fertilizantes e Defensivos' : 'fertilizantes_e_defensivos',
    'Químicos Diversos' : 'quimicos_diversos',
    'Madeira' : 'madeira',
    'Papel e Celulose' : 'papel_e_celulose',
    'Embalagens' : 'embalagens',
    'Materiais Diversos' : 'materiais_diversos',
    'Produtos para Construção' : 'produtos_para_construcao',
    'Construção Pesada' : 'construcao_pesada',
    'Engenharia Consultiva' : 'engenharia_consultiva',
    'Serviços Diversos' : 'servicos_diversos',
    'Material Aeronáutico e de Defesa' : 'material_aeronautico_e_de_defesa',
    'Material Rodoviário' : 'material_rodoviario',
    'Motores, Compressores e Outros' : 'motores_compressores_e_outros',
    'Máq. e Equip. Industriais' : 'maq._e_equip._industriais',
    'Máq. e Equip. Construção e Agrícolas' : 'maq._e_equip._construcao_e_agricolas',
    'Armas e Munições' : 'armas_e_municoes',
    'Transporte Aéreo' : 'transporte_aereo',
    'Transporte Ferroviário' : 'transporte_ferroviario',
    'Transporte Hidroviário' : 'transporte_hidroviario',
    'Transporte Rodoviário' : 'transporte_rodoviario',
    'Exploração de Rodovias' : 'exploracao_de_rodovias',
    'Serviços de Apoio e Armazenagem' : 'servicos_de_apoio_e_armazenagem',
    'Material de Transporte' : 'material_de_transporte',
    'Agricultura' : 'agricultura',
    'Açucar e Alcool' : 'acucar_e_alcool',
    'Carnes e Derivados' : 'carnes_e_derivados',
    'Alimentos Diversos' : 'alimentos_diversos',
    'Cervejas e Refrigerantes' : 'cervejas_e_refrigerantes',
    'Produtos de Uso Pessoal' : 'produtos_de_uso_pessoal',
    'Produtos de Limpeza' : 'produtos_de_limpeza',
    'Alimentos' : 'alimentos',
    'Incorporações' : 'incorporacoes',
    'Fios e Tecidos' : 'fios_e_tecidos',
    'Vestuário' : 'vestuario',
    'Calçados' : 'calcados',
    'Acessórios' : 'acessorios',
    'Eletrodomésticos' : 'eletrodomesticos',
    'Móveis' : 'moveis',
    'Utensílios Domésticos' : 'utensilios_domesticos',
    'Automóveis e Motocicletas' : 'automoveis_e_motocicletas',
    'Hotelaria' : 'hotelaria',
    'Restaurante e Similares' : 'restaurante_e_similares',
    'Bicicletas' : 'bicicletas',
    'Brinquedos e Jogos' : 'brinquedos_e_jogos',
    'Produção de Eventos e Shows' : 'producao_de_eventos_e_shows',
    'Viagens e Turismo' : 'viagens_e_turismo',
    'Atividades Esportivas' : 'atividades_esportivas',
    'Serviços Educacionais' : 'servicos_educacionais',
    'Aluguel de carros' : 'aluguel_de_carros',
    'Programas de Fidelização' : 'programas_de_fidelizacao',
    'Tecidos, Vestuário e Calçados' : 'tecidos_vestuario_e_calcados',
    'Produtos Diversos' : 'produtos_diversos',
    'Medicamentos e Outros Produtos' : 'medicamentos_e_outros_produtos',
    'Serviços Médico - Hospitalares, Análises e Diagnósticos' : 'servicos_medico_-_hospitalares_analises_e_diagnosticos',
    'Equipamentos' : 'equipamentos',
    'Computadores e Equipamentos' : 'computadores_e_equipamentos',
    'Programas e Serviços' : 'programas_e_servicos',
    'Telecomunicações' : 'telecomunicacoes',
    'Produção e Difusão de Filmes e Programas' : 'producao_e_difusao_de_filmes_e_programas',
    'Energia Elétrica' : 'energia_eletrica',
    'Água e Saneamento' : 'agua_e_saneamento',
    'Gás' : 'gas',
    'Bancos' : 'bancos',
    'Soc. Crédito e Financiamento' : 'soc._credito_e_financiamento',
    'Soc. Arrendamento Mercantil' : 'soc._arrendamento_mercantil',
    'Securitizadoras de Recebíveis' : 'securitizadoras_de_recebiveis',
    'Gestão de Recursos e Investimentos' : 'gestao_de_recursos_e_investimentos',
    'Serviços Financeiros Diversos' : 'servicos_financeiros_diversos',
    'Seguradoras' : 'seguradoras',
    'Corretoras de Seguros' : 'corretoras_de_seguros',
    'Exploração de Imóveis' : 'exploracao_de_imoveis',
    'Intermediação Imobiliária' : 'intermediacao_imobiliaria',
    'Holdings Diversificadas' : 'holdings_diversificadas',
    'Outros Títulos' : 'outros_titulos',
    'Outros' : 'outros'   
}

df_setorial['segmento'] = df_setorial['segmento'].map(setor_map)
setor_dummies = pd.get_dummies(df_setorial.segmento, prefix='segmento', prefix_sep='.').drop(columns='segmento.outros')
df_setorial = pd.concat([df_setorial, setor_dummies], axis=1).drop(columns='segmento')

df_setorial.drop(columns='listagem_segmento', inplace=True)


## TEST yfinance

In [32]:
tickers = [
    'TRPL4','CALI4','JFEN3','ENBR3','BAUH4','IRBR3','JHSF3','CRFB3',
    'PTNT3','JALL3','CAMB3','ALPA4','JFEN3','USIM5','BSLI3','RAIL3','CSAN3',
    'LREN3','EGIE3','SAPR11','CCPR3','BBAS3','MULT3','PDTC3','CESP6','BSEV3',
    'BMEB4','TOTS3','CGAS5','PINE4','BRSR6','SUZB3','KLBN11','TIMS3','AGRO3',
    'ROMI3','CSRN3','CEEB3','CEPE5','AERI3','LOGG3','EKTR4','NEOE3','AFLT3',
    'BPAC11','BPAN4','DTEX3','BRPR3','SMTO3','JFEN3','PSSA3','BBSE3','ABCB4',
    'BBDC4','SANB11','ITUB4','CMIN3','CIEL3','OPCT3','WEST3','BMOB3','JBDU3',
    'JALL3','POWE3','MBLY3','MOSI3','CAML3','ESPA3','INEP4','ORVR3','HBRE3',
    'ELMD3']

for ticker in tickers:
    print(f'---- {ticker} ----')
    df_history = yf.download(tickers=f"{ticker}.SA", period='2y')
    df_history['ticker'] = ticker
    print(df_history.shape)
    if(df_history.shape[0] >= 180):
        df_history.to_parquet(f'history/{ticker.lower()}.parquet')
    else:
        print('Data <- 180 days')
    print()    

In [35]:
# CONCAT EVERY DF
df_list = []
for data in os.listdir('history'):
    df_list.append(pd.read_parquet(f'history/{data}'))
df = pd.concat(df_list)
df = df.reset_index()
df.columns = ['date', 'open','high','low','close','adj_close','volume','ticker']

df['ticker'] = df['ticker'].str.slice(0,4)
df = df[['adj_close','date', 'ticker']]

df = df.merge(
        df_setorial,
        how='left',
        left_on='ticker',
        right_on='codigo')

tickers = df['ticker'].unique()

ticker_dummies = pd.get_dummies(df.ticker, prefix='ticker', prefix_sep='.')
df = pd.concat([df, ticker_dummies], axis=1).drop(columns='ticker')        

df = df.drop(columns='codigo')

df = df.sort_values(by=['date'], ascending=True)

df['date'] = df['date'].astype(str)
df['date'] = df['date'].str.replace('-','')
df['date'] = df['date'].astype(int)

df.reset_index(drop=True, inplace=True)

df = df.dropna()

df.head()

,adj_close,date,setor.bens_industriais,setor.comunicacoes,setor.consumo_ciclico,setor.consumo_nao_ciclico,setor.financeiro,setor.materiais_basicos,setor.petroleo_gas_e_biocombustiveis,setor.saude,...,ticker.PTNT,ticker.RAIL,ticker.ROMI,ticker.SANB,ticker.SAPR,ticker.SMTO,ticker.SUZB,ticker.TOTS,ticker.TRPL,ticker.USIM
0,16.925488,20190226,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15.296380,20190226,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27.370691,20190226,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34.487808,20190226,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7.480000,20190226,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
days_to_predict = 10
rows_to_predict = len(tickers) * days_to_predict

X_train = df.iloc[:-rows_to_predict,1:]
y_train = df.iloc[:-rows_to_predict,0]

X_test = df.iloc[-rows_to_predict:,1:]
y_test = df.iloc[-rows_to_predict:,0]


In [80]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train,y_train)
predicted = regressor.predict(X_test)

In [81]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print(f'r2_score : {round(r2_score(predicted, y_test),4)}')
print(f'mean_squared_error: {round(mean_squared_error(predicted, y_test, squared=False),4)}')
print(f'mean_absolute_error: {round(mean_absolute_error(predicted, y_test),4)}')

r2_score : 0.9925
mean_squared_error: 2.4988
mean_absolute_error: 1.4872
